In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os

# -------------------------
# Step 1. File selection (Tk file picker)
# -------------------------
root = Tk()
root.withdraw()  # hide root window

file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)
if not file_path:
    raise SystemExit("❌ No file selected.")
print(f"Selected file: {file_path}")

# -------------------------
# Step 2. Year & Month dialog (returns year int, month name)
# -------------------------
def ask_year_month():
    result = {"year": None, "month": None}

    def on_ok():
        result["year"] = year_var.get()
        result["month"] = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    # Year dropdown
    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))  # default
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    # Month dropdown
    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])  # default
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)

    win.wait_window()  # wait for dialog to close
    return int(result["year"]), result["month"]

selected_year, selected_month = ask_year_month()
print(f"Selected: {selected_month} {selected_year}")

# -------------------------
# Step 3. Read & normalize input
# -------------------------
print("⏳ Reading input file...")
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")

# normalize Period Month to first-of-month timestamps
df["Period Month"] = pd.to_datetime(df["Period Month"], errors="coerce").dt.to_period("M").dt.to_timestamp()

# ensure numeric FTE
df["FTE"] = pd.to_numeric(df["FTE"], errors="coerce")

# -------------------------
# Step 4. Build 24-month horizon
# -------------------------
sel_year = selected_year
sel_month = list(calendar.month_name).index(selected_month)  # numeric month
curr_date = datetime(sel_year, sel_month, 1)

start_date = datetime(sel_year, 1, 1)
horizon = pd.date_range(start=start_date, periods=24, freq="MS")
month_labels = [d.strftime("%b-%y") for d in horizon]

# -------------------------
# Step 5. Grouping and carry-forward logic
# -------------------------
output_records = []

def get_key(row):
    """Grouping key: prefer PID; otherwise use MPP ID. BF and GCB included."""
    pid = row.get("PID")
    mpp = row.get("MPP ID")
    gcb = row.get("GCB")
    bf = row.get("Business Framework")
    if pd.notna(pid) and str(pid).strip() != "":
        return ("PID", str(pid).strip(), str(gcb), str(bf))
    else:
        return ("MPP", str(mpp).strip() if pd.notna(mpp) else "", str(gcb), str(bf))

print("⏳ Processing records...")
grouped = df.groupby(df.apply(get_key, axis=1))

for key, group in grouped:
    # key: ("PID"/"MPP", id, GCB, BF)
    kind = key[0]
    group = group.sort_values("Period Month", ignore_index=True)

    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # initialize horizon months
    for label in month_labels:
        record[label] = None

    # last_fte holds the most recent known FTE (can be negative, e.g., -1)
    last_fte = None

    # iterate months in chronological horizon order
    for month_label in month_labels:
        month_dt = datetime.strptime(month_label, "%b-%y")

        # find any records in this month (if multiple, take last by order)
        rows_in_month = group[group["Period Month"] == month_dt]
        if not rows_in_month.empty:
            # use last record's FTE for that month
            val = rows_in_month["FTE"].iloc[-1]
            # val may be NaN; treat numeric -1 as valid leaver marker
            if pd.isna(val):
                # keep last_fte as-is (no change)
                pass
            else:
                last_fte = val

        # set monthly cell to last_fte if available (this will also set -1 if that's the last_fte)
        if last_fte is not None:
            record[month_label] = last_fte
        else:
            record[month_label] = None

        # For snapshot FTE: set from last known fte seen up to and including selected month
        if month_dt <= curr_date:
            # Important: use last_fte (so -1 is preserved), not record[month_label] only
            record["FTE"] = last_fte if last_fte is not None else None

    output_records.append(record)

# -------------------------
# Step 6. Build dataframe, sort and save
# -------------------------
out_df = pd.DataFrame(output_records)

# Sorting order
sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
# Ensure sort columns exist in dataframe
existing_sort_cols = [c for c in sort_cols if c in out_df.columns]
out_df = out_df.sort_values(existing_sort_cols, na_position="last").reset_index(drop=True)

# Save to Excel next to input file
out_file = os.path.join(
    os.path.dirname(file_path),
    f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx"
)
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

print(f"✅ ADWP Output written to: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns):,}")


In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os

# -------------------------
# Step 1. File selection
# -------------------------
root = Tk()
root.withdraw()  # hide root window

file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)
if not file_path:
    raise SystemExit("❌ No file selected.")
print(f"Selected file: {file_path}")


# -------------------------
# Step 2. Custom dialog for Year & Month
# -------------------------
def ask_year_month():
    result = {"year": None, "month": None}

    def on_ok():
        result["year"] = year_var.get()
        result["month"] = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    # Year dropdown
    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))  # default
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    # Month dropdown
    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])  # default
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)

    win.wait_window()  # wait for user to close

    return int(result["year"]), result["month"]


selected_year, selected_month = ask_year_month()
print(f"Selected: {selected_month} {selected_year}")


# -------------------------
# Step 3. Processing
# -------------------------
print("⏳ Reading input file...")
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")
df["Period Month"] = pd.to_datetime(df["Period Month"]).dt.to_period("M").dt.to_timestamp()

# Current selection
sel_year = selected_year
sel_month = list(calendar.month_name).index(selected_month)
curr_date = datetime(sel_year, sel_month, 1)

# Horizon (24 months from Jan of selected year)
start_date = datetime(sel_year, 1, 1)
horizon = pd.date_range(start=start_date, periods=24, freq="MS")
month_labels = [d.strftime("%b-%y") for d in horizon]

output_records = []

def get_key(row):
    if pd.notna(row["PID"]):
        return ("PID", row["PID"], row["GCB"], row["Business Framework"])
    else:
        return ("MPP", row["MPP ID"], row["GCB"], row["Business Framework"])

print("⏳ Processing records...")
grouped = df.groupby(df.apply(get_key, axis=1))

for key, group in grouped:
    kind = key[0]
    group = group.sort_values("Period Month")

    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # initialize horizon columns
    for label in month_labels:
        record[label] = None

    last_fte = None
    for month in month_labels:
        month_dt = datetime.strptime(month, "%b-%y")
        row = group[group["Period Month"] == month_dt]
        if not row.empty:
            last_fte = row["FTE"].iloc[0]
        if last_fte is not None:
            record[month] = last_fte
        if month_dt <= curr_date and record[month] is not None:
            record["FTE"] = record[month]

    output_records.append(record)

# Build DataFrame
out_df = pd.DataFrame(output_records)

# Sorting
sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
out_df = out_df.sort_values(sort_cols, na_position="last").reset_index(drop=True)

# Save output
out_file = os.path.join(
    os.path.dirname(file_path),
    f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx"
)
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

print(f"✅ ADWP Output written to: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns)}")
